In [ ]:
!pip install transformers accelerate evaluate datasets peft -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import transformers
import accelerate
import peft

print(f"Transformers version: {transformers.__version__}")
print(f"Accelerate version: {accelerate.__version__}")
print(f"PEFT version: {peft.__version__}")

Transformers version: 4.46.2
Accelerate version: 1.1.1
PEFT version: 0.13.2


In [ ]:
# model_checkpoint = "google/vit-base-patch16-224-in21k"
model_checkpoint = "google/vit-base-patch16-224"

In [ ]:
from datasets import load_dataset

dataset = load_dataset("food101", split="train[:5000]")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

train-00000-of-00008.parquet:   0%|          | 0.00/490M [00:00<?, ?B/s]

train-00001-of-00008.parquet:   0%|          | 0.00/464M [00:00<?, ?B/s]

train-00002-of-00008.parquet:   0%|          | 0.00/472M [00:00<?, ?B/s]

...

validation-00000-of-00003.parquet:   0%|          | 0.00/423M [00:00<?, ?B/s]

validation-00001-of-00003.parquet:   0%|          | 0.00/413M [00:00<?, ?B/s]

validation-00002-of-00003.parquet:   0%|          | 0.00/426M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/75750 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/25250 [00:00<?, ? examples/s]

In [ ]:
labels = dataset.features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

id2label[2]

'baklava'

In [ ]:
from transformers import AutoImageProcessor

image_processor = AutoImageProcessor.from_pretrained(model_checkpoint)

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


In [ ]:
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    Resize,
    ToTensor,
)

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
train_transforms = Compose(
    [
        RandomResizedCrop(image_processor.size["height"]),
        RandomHorizontalFlip(),
        ToTensor(),
        normalize,
    ]
)

val_transforms = Compose(
    [
        Resize(image_processor.size["height"]),
        CenterCrop(image_processor.size["height"]),
        ToTensor(),
        normalize,
    ]
)


def preprocess_train(example_batch):
    """Apply train_transforms across a batch."""
    example_batch["pixel_values"] = [train_transforms(image.convert("RGB")) for image in example_batch["image"]]
    return example_batch


def preprocess_val(example_batch):
    """Apply val_transforms across a batch."""
    example_batch["pixel_values"] = [val_transforms(image.convert("RGB")) for image in example_batch["image"]]
    return example_batch

In [ ]:
splits = dataset.train_test_split(test_size=0.1)
train_ds = splits["train"]
val_ds = splits["test"]

In [ ]:
train_ds.set_transform(preprocess_train)
val_ds.set_transform(preprocess_val)

In [ ]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )

In [ ]:
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer

model = AutoModelForImageClassification.from_pretrained(
    model_checkpoint,
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes=True,  # provide this in case you're planning to fine-tune an already fine-tuned checkpoint
)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([101]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([101, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print_trainable_parameters(model)

trainable params: 85876325 || all params: 85876325 || trainable%: 100.00


In [ ]:
for name, module in model.named_modules():
    print(name, ":", module)

 : ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTSdpaAttention(
            (attention): ViTSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in

In [ ]:
import re
# def extract_target_layers(model):
#     """
#     Extracts layers matching attention blocks like:
#     'encoder.layer.X.attention.attention.query',
#     'encoder.layer.X.attention.attention.key',
#     'encoder.layer.X.attention.attention.value'
#     """
#     # Adjust the pattern based on actual printed names
#     pattern = re.compile(r'vit\.encoder\.layer\.\d+\.attention\.attention\.(query|key|value)')

#     # Extract matching layers
#     target_layers = [name for name, _ in model.named_parameters() if pattern.search(name)]

#     return target_layers

# # Extract the target layers
# target_layers = extract_target_layers(model)
# print(target_layers)

# def extract_target_layers(model):
#     """
#     Extracts layer names matching the pattern:
#     'vit.encoder.layer.X.attention.attention.query'
#     'vit.encoder.layer.X.attention.attention.key'
#     'vit.encoder.layer.X.attention.attention.value'
#     without the '.weight' or '.bias' suffixes.
#     """
#     # Define the regex pattern to match only the base layer names
#     pattern = re.compile(r'(vit\.encoder\.layer\.\d+\.attention\.attention\.(query|key|value))')

#     # Use a set to avoid duplicates
#     target_layers = set()

#     # Extract matching layers from model parameters
#     for name, _ in model.named_parameters():
#         match = pattern.match(name)
#         if match:
#             # Add only the base layer name (without '.weight' or '.bias')
#             target_layers.add(match.group(1))

#     # Convert set to list and return
#     return list(target_layers)

def extract_encoder_layers(model):
    """
    Extracts layer names matching the pattern:
    'encoder.*'
    without the '.weight' or '.bias' suffixes.
    """
    # Define the regex pattern to match all encoder layers
    pattern = re.compile(r'(vit\.encoder\..+?)\.(weight|bias)')

    # Use a set to avoid duplicates
    encoder_layers = set()

    # Extract matching layers from model parameters
    for name, _ in model.named_parameters():
        match = pattern.match(name)
        if match:
            # Add only the base layer name (without '.weight' or '.bias')
            encoder_layers.add(match.group(1))

    # Convert set to list and return
    return list(encoder_layers)


# Extract the target layers
target_layers = [i for i in extract_encoder_layers(model) if not 'layernorm' in i]
target_layers = [i for i in target_layers if not 'key' in i]
target_layers = [i for i in target_layers if not 'dense' in i]

# Print the extracted layers
print(target_layers)

['vit.encoder.layer.1.attention.attention.value', 'vit.encoder.layer.11.attention.attention.value', 'vit.encoder.layer.2.attention.attention.query', 'vit.encoder.layer.0.attention.attention.value', 'vit.encoder.layer.5.attention.attention.value', 'vit.encoder.layer.9.attention.attention.value', 'vit.encoder.layer.8.attention.attention.value', 'vit.encoder.layer.6.attention.attention.query', 'vit.encoder.layer.1.attention.attention.query', 'vit.encoder.layer.9.attention.attention.query', 'vit.encoder.layer.3.attention.attention.value', 'vit.encoder.layer.3.attention.attention.query', 'vit.encoder.layer.7.attention.attention.value', 'vit.encoder.layer.5.attention.attention.query', 'vit.encoder.layer.4.attention.attention.value', 'vit.encoder.layer.8.attention.attention.query', 'vit.encoder.layer.10.attention.attention.value', 'vit.encoder.layer.0.attention.attention.query', 'vit.encoder.layer.11.attention.attention.query', 'vit.encoder.layer.10.attention.attention.query', 'vit.encoder.la

In [ ]:
# from peft import LoraConfig, get_peft_model

# config = LoraConfig(
#     r=4,
#     lora_alpha=16,
#     target_modules=["query", "key", "value", "dense"],
#     lora_dropout=0.1,
#     bias="none",
#     modules_to_save=["classifier"],
# )
# lora_model = get_peft_model(model, config)
# print_trainable_parameters(lora_model)

from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=8,
    target_modules = target_layers,
    lora_dropout=0.1,
    bias="none",
    modules_to_save=["classifier"],
)
lora_model = get_peft_model(model, config)
print_trainable_parameters(lora_model)

trainable params: 372581 || all params: 86248906 || trainable%: 0.43


In [ ]:
import numpy as np
import evaluate

# Load the evaluation metrics
accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    """Computes accuracy, precision, recall, and F1 score on a batch of predictions."""
    # Get the predictions and the true labels
    predictions = np.argmax(eval_pred.predictions, axis=1)
    references = eval_pred.label_ids

    # Compute each metric
    accuracy = accuracy_metric.compute(predictions=predictions, references=references)
    precision = precision_metric.compute(predictions=predictions, references=references, average='weighted')
    recall = recall_metric.compute(predictions=predictions, references=references, average='weighted')
    f1 = f1_metric.compute(predictions=predictions, references=references, average='weighted')

    # Return all metrics as a dictionary
    return {
        "accuracy": accuracy["accuracy"],
        "precision": precision["precision"],
        "recall": recall["recall"],
        "f1": f1["f1"]
    }

In [ ]:
import torch


def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

In [ ]:
import torch
from transformers import TrainingArguments, Trainer

# Ensure the model is loaded on the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model to GPU
# lora_model = lora_model.to(device)

model = model.to(device)

model_name = model_checkpoint.split("/")[-1]
batch_size = 128

args = TrainingArguments(
    f"{model_name}-finetuned-lora-food101",
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-3,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    fp16=torch.cuda.is_available(),  # Use mixed precision only if GPU is available
    num_train_epochs=5,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
    label_names=["labels"],
    disable_tqdm=False,  # Disable tqdm progress bar
)

trainer = Trainer(
    lora_model,
    args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
import wandb

# Initialize wandb
wandb.init(
    project="vit-base-finetuning-food101-key-value_LoRA",
    group="LoRA-vs-Base",
    name=f"{model_checkpoint}-run-{wandb.util.generate_id()}_Rank8",
    config=args
)

eval/accuracy,▁▄██▇
eval/f1,▁▄██▇
eval/loss,█▁▁▁▁
eval/precision,▁▄██▇
eval/recall,▁▄██▇
eval/runtime,█▄▁▂▄
eval/samples_per_second,▁▅█▇▅
eval/steps_per_second,▁▅█▇▅
train/epoch,▁▁▃▃▅▅▆▇██
train/global_step,▁▁▃▃▅▅▆▇██
train/grad_norm,█▂▅▁


In [ ]:
train_results = trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.121931,0.948000,0.948533,0.948000,0.947772
2,1.016800,0.082108,0.968000,0.967992,0.968000,0.967892
3,0.207500,0.080491,0.972000,0.972737,0.972000,0.972008
4,0.155100,0.084355,0.966000,0.967716,0.966000,0.966065
5,0.146300,0.082107,0.972000,0.972502,0.972000,0.971997


In [ ]:
trainer.evaluate(val_ds)

{'eval_loss': 0.16093569993972778,
 'eval_accuracy': 0.956,
 'eval_precision': 0.9600420372832196,
 'eval_recall': 0.956,
 'eval_f1': 0.9566763409328298,
 'eval_runtime': 4.5484,
 'eval_samples_per_second': 109.929,
 'eval_steps_per_second': 0.879,
 'epoch': 5.0}

In [ ]:
# import os
# os.environ["WANDB_DISABLED"] = "true"

In [ ]:
# import os
# import logging
# from transformers import TrainingArguments, Trainer

# # Enable logging and tqdm
# logging.basicConfig(level=logging.INFO)
# from transformers.utils import logging as hf_logging
# hf_logging.set_verbosity_info()

# # Ensure the model is loaded on the GPU if available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = model.to(device)

# model_name = model_checkpoint.split("/")[-1]
# batch_size = 128

# args = TrainingArguments(
#     f"{model_name}-finetuned-lora-food101",
#     remove_unused_columns=False,
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     learning_rate=5e-3,
#     per_device_train_batch_size=batch_size,
#     gradient_accumulation_steps=4,
#     per_device_eval_batch_size=batch_size,
#     fp16=torch.cuda.is_available(),
#     num_train_epochs=5,
#     logging_steps=10,
#     load_best_model_at_end=True,
#     metric_for_best_model="accuracy",
#     push_to_hub=False,
#     label_names=["labels"],
#     disable_tqdm=False,  # Ensure tqdm is enabled
# )

# trainer = Trainer(
#     model,
#     args,
#     train_dataset=train_ds,
#     eval_dataset=val_ds,
#     tokenizer=image_processor,
#     compute_metrics=compute_metrics,
#     data_collator=collate_fn,
# )

In [ ]:
# train_results = trainer.train()

{'eval_loss': 0.14475855231285095,
 'eval_accuracy': 0.96,
 'eval_runtime': 3.5725,
 'eval_samples_per_second': 139.958,
 'eval_steps_per_second': 1.12,
 'epoch': 5.0}